# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file and a CSV file (same content, different formats), which is an array of objects that look like this:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "d": 1593820800, // timestamp
    "p": 0,          // involves a pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist? (1=true, 0=false)
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "s": 0,          // crash severity (O=no injuries, A=any injury, K=fatal)
}
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [ ]:
import pandas as pd

In [ ]:
exports = ['165957']

### Import roles from table 2

In [ ]:
roles = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Person Type Text Format']
    )
    roles.append( df )

roles_df = pd.concat(roles).groupby(['CrashId'])['Person Type Text Format'].unique() # .groupby(['CrashId'])['Person Type Text Format'].unique()

print(type(roles_df))


print(roles_df)

### Import crashes from table 0

In [ ]:
localData = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Crash Severity', 'Route Class',
                 'Roadway Name', 'Intersecting Roadway Name']
    )
    localData.append( df )

print(localData)

localData_df = pd.concat(localData)

print(type(localData_df))

crashesTable0_df = localData_df.merge(roles_df, on='CrashId', how='inner')

print(crashesTable0_df)

Import crashes from table 1


In [ ]:
vehicleData = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_1.csv',
        skiprows = 1,
        usecols=['CrashId', 'Direction of Travel Before Crash', 'Contributing Circumstances, Motor Vehicle', 
                 'Posted/Statutory Speed Limit', 'Bike Lanes/Sharrows Present']
    )
    vehicleData.append ( df )

crashesTable1_df = pd.concat(vehicleData)

# crashesTableFinal = pd.concat([crashesTable2_df, crashesTable1_df, crashesTable2_df], axis=1, verify_integrity=True)

crashesTableFinal_df = crashesTable0_df.merge(crashesTable1_df, on='CrashId', how='inner')

### Merge data and save to .json and .csv

In [ ]:
crashesTableFinal_df.to_csv('crashes.csv', index=False)

crashesTableFinal_df.head()


In [ ]:
# Rename columns to make JSON output smaller
crashesTableFinal_df.columns = ['id', 'x', 'y', 'd', 't', 's', 'r', 'o', 'h',\
    'k', 'j', 'm', 'b', 'v']

crashesTableFinal_df.head()

In [ ]:
# Transform timestamps
crashesTableFinal_df.d = crashesTableFinal_df.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashesTableFinal_df.t = crashesTableFinal_df.t.apply(lambda x: x[:-3])

# Set up pedestrian and cyclist flags
crashesTableFinal_df['p'] = crashesTableFinal_df.id.apply(lambda x: 1 if x in roles_df and 'Pedestrian' in roles_df[x] else 0)
crashesTableFinal_df['c'] = crashesTableFinal_df.id.apply(lambda x: 1 if x in roles_df and 'Bicyclist' in roles_df[x] else 0)

crashesTableFinal_df.head()

In [ ]:
# Save as JSON
crashesTableFinal_df.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashesTableFinal_df.to_csv('crashes.csv', index=False)